## 1. 前置知识


### 1.1 KL散度
KL散度是一种衡量概率分布间差异的方式，公式如下：  
$$
D_{KL}(P \parallel Q) = \sum_{i} P(i) \log \left( \frac{P(i)}{Q(i)}  \right)
$$
概率P与概率Q之间的差异，KL散度越小，则两个概率越接近，那么估计的概率分布也就与真实的概率分布越接近 </br>
一个简单的例子</br>
<img src="asset/binomial_uniform_distribution.png" width="600"  alt="dists">
| | 0 | 1 | 2|
|-------|-------|-------|-------|
|P | 0.36| 0.48| 0.16|
|Q | 0.333| 0.333| 0.333|

则KL散度的计算流程如下：  
$$
    D_{KL}(Q \parallel P) = \sum_{i} \ln \left( \frac{Q(i)}{P(i)} \right) \\
    = 0.333 \ln(\frac{0.333}{0.36}) + 0.333 \ln(\frac{0.333}{0.48}) + 0.333 \ln(\frac{0.333}{0.16}) \\
    = -0.02596 + (-0.12176) + 0.24408 \\
    = 0.09637
$$
### 1.2 高斯混合模型
高斯混合模型：用若干个高斯分布的叠加去近似逼近数据分布

<img src="asset/GMM.png" width="600"  alt="dists">

## 2. VAE原理
首先从Auto Encoder说起，AE对图像进行encode以后，生成一个编码图片信息的向量，然后再通过一个decoder将原始的图片还原回来。具体流程如下  

```mermaid
graph LR
    A[Images] --> B[Encoder]
    B --> C[Latent vector]
    C --> D[Decoder]
    D --> E[Reconstructed Images]
```

但这里有个问题，AE结构只能对图像进行压缩和还原，并不能生成新的图像，怎么解决该问题呢？这就需要将latent概率化，这样就可以在特定的概率分布中获取一定的随机性。
<img src="asset/vae_demo.jpg" width="600"  alt="dists">



VAE将latent表达为高斯的概率分布，同时通过网络去自动学习平衡图像生成的精准度和概率分布的拟合度，这样两者就可以分别用MSE/BCE和KL散度来计算。之所以使用高斯分布，是因为高斯混合模型可以去拟合任何的数据分布，同时高斯分布可以通过参数重整化转化成标准正态分布的线性表达，因此VAE中的latent，包含了高斯分布的均值和标准差，具体流程图如下：

```mermaid
graph LR
    A[Images] --> B[Encoder]
    B --> C[mean]
    B --> D[std]
    C --> E[sampled latent]
    D --> E
    E --> F[Decoder]
    F --> G[Reconstructed Images]
```

<img src="asset/vae_workflow.png" width="600"  alt="dists">

<img src="asset/vae_demo1.jpg" width="600"  alt="dists">

## 3. 结构图
<img src="asset/vae_struct.png" width="600"  alt="dists">   

在AE中，编码器是直接产生一个编码的，在VAE中，为了给编码添加合适的噪声，编码器会输出两个编码，一个是原有编码(m1, m2, m3), 另一个是控制噪声干扰程度的编码(sigma1, sigma2, sigma3), 第二个编码其实很好理解，就是为随机噪声码(e1, e2, e3)分配权重，然后加上exp(simga_i)的目的是为了保证这个分配的权重是个正值，最后将原编码与噪声编码相加，就得到了VAE在encode层的输出结果(c1, c2, c3)。  
损失函数方面，除了必要的重构损失外，VAE还添加了一个损失函数，这同样是必要的部分，因为如果不加的话，整个模型就会出现问题：为了保证生成图片的质量越高，编码器肯定希望噪声对自身生成图片的干扰越小，于是分配给噪音的权重越小，这样只需要将(sigma1, sigma2, sigma3)赋为接近无穷大的值就好了。所以，第二个损失函数就有限制编码器走这样极端路径的作用，这也从直观上就能看出来，exp(\sigma_i) * (1+\sigma_i)在sigma_i=0处取得最小值，于是(sigma1, sigma2, sigma3)就会避免被赋值为负无穷大。

### 6. 参考资料
1. [读研学习笔记：VAE](https://www.xiaohongshu.com/explore/66728ca70000000006007aa7)
2. [公式推导, 还在头疼Diffusion模型公式吗？](https://www.bilibili.com/video/BV1Ax4y1v7CY/)